# DSCI 100 Project: Final Report

**Authors**: 

**Predicting Usage of a Video Game Research Server**
A research group in the Department of Computer Science at UBC (PLAI) has a goal to aid in advancing the limits of artificial intelligence. They host a Minecraft server that records players' actions as they navigate through the world. Since running the project is so complex, they want to recruit players who will contribute large amounts of gameplay data, as well as make sure they have the resources to handle the amount of players they attract. In an attempt to optimize their players' contributions, they want to understand what demographics of players will have the greatest gameplay time. This project answers their question of how player demographics relate to total gameplay time. 

## The Question
**Question 2**: Which kinds of players are most likely to contribute a large amount of data, and how can we identify them for targeted recruitment?

**Predictive Question** Can we predict the total `played_hours` from the `players.csv` dataset based on the two predictors, `experience` and `subscribe`.

**Response variable**:

`played_hours` - total play time and data contribution

**Explanatory variables**:

`experience` - player experience level

`subscribe` -  subscription status of player

The dataset allows us to examine whether `experience` or `subscribe` relates to total playtime. If certain experience levels or subscribtion status tend to play more, those groups may contribute more gameplay data. This helps identify the player demographics most likely to provide substantial data for the research team.


## # Import libraries
import pandas as pd
import altair as alt

# import the K-NN regression model
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import set_config

# import the K-NN regression model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_errorData Description: The Players
The dataset used in this analysis is a player demographics and engagement dataset that contains 196 observations (rows) and 9 variables (columns). The dataset records information about individual players, their gaming experience level, subscription status, demographic details, and the amount of time they have spent playing.

- Number of observations: 196 (each row represents one unique player)
- Number of variables: 9
- Observational unit: Player-level data
- Purpose: Identify which demographic characteristics correspond to higher gameplay time (and therefore more data contributed)
  
**Variables**
- `played_hours` (numeric) - Total hours each player spent on the server
- `experience` (categorical) -  Self-reported experience level in Minecraft
- `gender` (categorical) - Player’s gender
- `subscribe` (Boolean) - Subscription status of player
- `hashedEmail` (String) - The players email, hashed for privacy
- `name` (String) - Name of player
- `age` (Numeric) - Age of player
- `individualID` (String) - Unique ID of player
- `organizationName` (String) - Organization name

**Issues to consider**
- Missing identifiers (`individualId`, `organizationName`).
- `hashedEmail` is anonymous for privacy.
- `played_hours` may include idle time, which could overestimate actual engagement.

Source of the original dataset:
https://drive.google.com/file/d/1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz/edit


In [2]:
# Import libraries
import pandas as pd
import altair as alt

# import the K-NN regression model
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import set_config

# import the K-NN regression model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Read the datasets
url= "https://raw.githubusercontent.com/sydlpeters/dsci-group-2025w1-group-101-1/refs/heads/main/data/players.csv"
# Load the dataset 
players = pd.read_csv(url)
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


In [3]:
# Drop columns that are entirely missing(NaN)
players_tidy = players.drop(columns=["individualId", "organizationName", "name", "gender", "hashedEmail", "age"])
# Preview tidy players_tidy
players_tidy.head()

# Change variables to numeric here

# Split here

,experience,subscribe,played_hours
0,Pro,True,30.3
1,Veteran,True,3.8
2,Veteran,False,0.0
3,Amateur,True,0.7
4,Regular,True,0.1


In [4]:
players_tidy.describe()

,played_hours
count,196.000000
mean,5.845918
std,28.357343
min,0.000000
25%,0.000000
50%,0.100000
75%,0.600000
max,223.100000


**Summary Statistics**

Most players recorded very little playtime, with three-quarters spending under an hour in total (Q3 = 0.6 hours). Only a small number logged substantially more time, which raises the overall average (mean = 5.85 hours; median = 0.1 hours) and affects the dataset (STD = 28.36 hours). **Most players played very little (close to 0 hours), but a few played way more, making the dataset highly uneven.**

In [5]:
# Experienced comapred with Average played hours played

# Summary statistics and graphs should only be done on training data

plot1= alt.Chart(players, title="Fig 1. Total Hours Played per Experience Group").mark_bar().encode(
    x=alt.X("experience:N").title("Experience Level"),
    y=alt.Y("mean(played_hours):Q").title("Average hours Played(hr)"),
    color = alt.Color("experience:N").title("Experience Level")
)
plot1

alt.Chart(...)

In [6]:
# Experienced comapred with Average played hours played
plot1= alt.Chart(players, title="Fig. 2 Average Hours Played per Experience Group").mark_bar().encode(
    x=alt.X("experience:N").title("Experience Level"),
    y=alt.Y("played_hours:Q").title("Average hours Played(hr)"),
    color = alt.Color("experience:N").title("Experience Level")
)
plot1

alt.Chart(...)

In [7]:
# Subscription status comapred with Average hours played
plot2= alt.Chart(players, title="Fig. 3 Average Hours by Subscribed/Unsubscribed").mark_bar().encode(
    x=alt.X("subscribe:N").title("Subscription Status"),
    y=alt.Y("mean(played_hours)").title("Average Hours Played(hr)"),
    color = alt.Color("subscribe:N").title("Subscription Status")
)
plot2

alt.Chart(...)

In [8]:
plot3= alt.Chart(players, title="Fig. 4 Average Hours Played by Gender").mark_bar().encode(
    x=alt.X("gender:N").title("Gender"),
    y=alt.Y("mean(played_hours)").title("Average Hours Played(hr)"),
    color = alt.Color("gender:N", legend=None).title("Gender")
)
plot3

alt.Chart(...)

In [9]:
plot4= alt.Chart(players, title="Fig. 5 Average Hours Played by each Age").mark_circle(opacity=0.8).encode(
    x=alt.X("age:N", scale=alt.Scale(domain=[i for i in range(2, 102, 2)])).title("Age"),
    y=alt.Y("mean(played_hours)").title("Average Hours Played(hr)"),
    color = alt.Color("age:N", legend=None).title("Age")
)
plot4

alt.Chart(...)

In [10]:
experience_map = {
    "Beginner" : 0,
    "Regular": 1,
    "Amateur": 2,
    "Veteran": 3,
    "Pro": 4
}
# Put above
players["experience"] = players["experience"].map(experience_map)
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,4,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,3,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,3,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,2,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,1,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,2,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,3,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,2,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,2,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


In [11]:
subscribe_map = {
    False : 0,
    True : 1,
}

players_tidy["subscribe"] = players_tidy["subscribe"].map(subscribe_map)
players_tidy

,experience,subscribe,played_hours
0,Pro,1,30.3
1,Veteran,1,3.8
2,Veteran,0,0.0
3,Amateur,1,0.7
4,Regular,1,0.1
...,...,...,...
191,Amateur,1,0.0
192,Veteran,0,0.3
193,Amateur,0,0.0
194,Amateur,0,2.3


In [12]:
# split data!
players_training, players_testing = train_test_split(
    players, test_size=0.2, random_state=1234
)

# set target and predictors
players_x_train = players_training[["experience"]]
players_y_train = players_training["played_hours"]

players_x_test = players_testing[["experience"]]
players_y_test = players_testing["played_hours"]

players_training

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
114,0,True,ae8d8a9dcf80b38466f89466201e6d594eb94d6994eda1...,1.0,Ella,Female,17,NaN,NaN
187,2,True,e3f0ad9aadd27f3d1d9197e58546d045018daa76767503...,0.0,Jasper,Male,17,NaN,NaN
39,2,True,6ef84a204d64edc3b3779127b13f298d0c70f96568486f...,0.0,Vivian,Male,17,NaN,NaN
25,1,True,5baba1651a0b92788bc0d6dcdf00be64af1cf9f0015bbe...,0.6,Kendall,Female,28,NaN,NaN
131,2,False,11bf6125c4264b3a8f3bffa57b33bd598e2ea1ecd6331a...,0.0,Olivia,Female,23,NaN,NaN
...,...,...,...,...,...,...,...,...,...
152,0,True,aea049eaa7cb10db386a62990220d205ceb2a4c473cae3...,0.2,Aarav,Prefer not to say,17,NaN,NaN
116,3,False,58893f3187db90bf0690c88f06e6aa5f8ea9ff9691ca8e...,0.0,Noah,Prefer not to say,20,NaN,NaN
53,2,True,fc0224c81384770e93ca717f32713960144bf0b52ff676...,0.2,Gemna,Male,27,NaN,NaN
38,3,True,d782933acd14c834e53dea816005a3583cb87710f7347a...,0.0,Ishaan,Male,17,NaN,NaN


In [14]:
# preprocess the data, make the pipeline
players_preprocessor = make_column_transformer(
    (
        StandardScaler(),
        [
            "experience",
            ],
    )
)
players_pipeline = make_pipeline(players_preprocessor, KNeighborsRegressor())

players_cv = pd.DataFrame(
    cross_validate(
        players_pipeline,
        players_x_train,
        players_y_train,
        cv=5,
        scoring="neg_root_mean_squared_error",
        return_train_score=True,
    )
)
players_cv

,fit_time,score_time,test_score,train_score
0,0.007018,0.001995,-9.099486,-21.719026
1,0.002997,0.000997,-5.920221,-22.051516
2,0.002001,0.001000,-28.129594,-16.957817
3,0.001998,0.001022,-1.713133,-22.352437
4,0.001001,0.000999,-32.491334,-15.254916


In [ ]:
# create the 5-fold GridSearchCV object
param_grid = {
    "kneighborsregressor__n_neighbors": range(1, 50, 1),
}

players_tuned = GridSearchCV(
    estimator=players_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
)

# fit the GridSearchCV object
players_result = pd.DataFrame(players_tuned.fit(players_x_train, players_y_train).cv_results_)

players_result

/opt/conda/lib/python3.11/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsregressor__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003402,0.000255,0.002116,0.000136,1,{'kneighborsregressor__n_neighbors': 1},-9.122226,-8.723420,-30.005672,-12.052734,-29.268655,-17.834541,9.707952,44
1,0.004764,0.002951,0.002155,0.000431,2,{'kneighborsregressor__n_neighbors': 2},-9.143509,-5.170949,-29.994092,-6.304415,-30.493426,-16.221278,11.523310,19
2,0.019322,0.032565,0.001978,0.000134,3,{'kneighborsregressor__n_neighbors': 3},-31.379840,-4.345652,-30.140029,-4.925939,-32.622795,-20.682851,13.127151,49
3,0.003059,0.000040,0.001914,0.000008,4,{'kneighborsregressor__n_neighbors': 4},-23.631453,-3.934581,-30.069712,-4.316454,-32.123221,-18.815084,12.317550,48
4,0.003074,0.000072,0.001927,0.000023,5,{'kneighborsregressor__n_neighbors': 5},-19.160247,-5.803200,-29.223031,-3.798582,-31.885128,-17.974038,11.580485,46
5,0.003063,0.000048,0.001899,0.000005,6,{'kneighborsregressor__n_neighbors': 6},-16.380491,-4.982276,-29.339701,-3.162105,-31.815267,-17.135968,11.899284,39
6,0.003034,0.000026,0.001914,0.000021,7,{'kneighborsregressor__n_neighbors': 7},-14.734910,-4.539553,-28.152575,-5.217285,-31.784133,-16.885692,11.332351,37
7,0.003272,0.000469,0.001898,0.000009,8,{'kneighborsregressor__n_neighbors': 8},-13.336478,-14.027496,-28.313692,-4.558332,-31.774824,-18.402164,10.134269,47
8,0.003083,0.000118,0.001906,0.000038,9,{'kneighborsregressor__n_neighbors': 9},-12.357332,-12.422075,-28.443800,-4.079793,-31.773035,-17.815207,10.538727,43
9,0.003062,0.000036,0.001906,0.000020,10,{'kneighborsregressor__n_neighbors': 10},-13.827831,-11.185561,-28.565624,-3.665939,-31.778249,-17.804641,10.682608,42


In [ ]:
# Add Visualization to compare the ks

In [ ]:
# Retrieve the CV scores


# get the best parameter values
players_min = players_tuned.best_params_
players_min

{'kneighborsregressor__n_neighbors': 39}

In [ ]:
players_best_RMSPE = -players_tuned.best_score_
players_best_RMSPE

np.float64(16.028531937252072)

In [16]:
knn = KNeighborsRegressor(n_neighbors = 39)
player_pipe = make_pipeline(players_preprocessor, knn)
player_pipe.fit(players_x_train, players_y_train)

players_predictions = players_testing.assign(prediction = player_pipe.predict(players_x_test))
players_predictions

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName,prediction
101,2,True,25879aecc205544bc6505f9faf768356e0a3b712605730...,0.0,Eli,Female,17,NaN,NaN,8.051282
51,1,True,b622593d2ef8b337dc554acb307d04a88114f2bf453b18...,218.1,Akio,Non-binary,20,NaN,NaN,6.846154
146,4,True,5669c0f4b50dbfe3e2f851e4bb89fa43c55cd1f71ba362...,0.0,Padma,Non-binary,25,NaN,NaN,1.305128
153,0,True,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,0.1,Osiris,Male,17,NaN,NaN,1.410256
106,1,False,63139b524e44d156daf265875fb8078bb2b24d6b7520a3...,0.0,Elias,Male,23,NaN,NaN,6.846154
59,3,True,ca20f724571080b997e0efa874b9611e9f280c1af5f68f...,0.2,Edward,Male,38,NaN,NaN,0.461538
161,3,False,f174555ae3ff613d5d0b98f83f26f689619d92ef272f28...,0.0,Finley,Non-binary,17,NaN,NaN,0.461538
167,0,False,42eafe96ed5c1684e3b5cc614d1b01a117173d3ec6898a...,0.3,Ariana,Female,17,NaN,NaN,1.410256
193,2,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN,8.051282
88,0,True,a1094a440899e69e804f888f44d1154ec0b7675d05d977...,0.0,Maya,Female,17,NaN,NaN,1.410256


In [ ]:

players_plot = (alt.Chart(players_predictions).mark_circle(opacity=0.7).encode(
    x=alt.X("experience").title("Experience").scale(zero=False),
    y=alt.Y("played_hours").title("played_hours").scale(zero=False)
)+
alt.Chart(
    players_predictions,
    title= "K=39"
).mark_line(
    color="black"
).encode(
    x="experience",
    y="prediction"
))
players_plot

# RMSE on the training add.

alt.LayerChart(...)

In [ ]:
players_predictions = players_predictions.rename(columns={"prediction":"prediction_80"})

players_cv = pd.DataFrame(
    cross_validate(
        players_pipeline,
        players_x_train,
        players_y_train,
        cv=5,
        scoring="neg_root_mean_squared_error",
        return_train_score=True,
    )
)

# create the 5-fold GridSearchCV object
param_grid = {
    "kneighborsregressor__n_neighbors": range(1, 50, 1),
}

players_tuned = GridSearchCV(
    estimator=players_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
)



players_result = pd.DataFrame(players_tuned.fit(players[["experience"]], players[["played_hours"]]).cv_results_)

players_result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsregressor__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003600,0.000201,0.003483,0.002256,1,{'kneighborsregressor__n_neighbors': 1},-9.021890,-50.832263,-27.077761,-10.159926,-31.266378,-25.671644,15.389364,37
1,0.003519,0.000046,0.002358,0.000055,2,{'kneighborsregressor__n_neighbors': 2},-9.080436,-50.837095,-24.783985,-10.089548,-29.553139,-24.868840,15.257793,9
2,0.003441,0.000018,0.002311,0.000016,3,{'kneighborsregressor__n_neighbors': 3},-30.152943,-50.825858,-24.381665,-10.132607,-29.235285,-28.945672,13.077045,49
3,0.003602,0.000213,0.003073,0.001017,4,{'kneighborsregressor__n_neighbors': 4},-23.354805,-50.673516,-24.220788,-10.135939,-29.112556,-27.499521,13.183768,48
4,0.003428,0.000038,0.002312,0.000024,5,{'kneighborsregressor__n_neighbors': 5},-19.522044,-50.621259,-24.146579,-10.105220,-29.056372,-26.690295,13.499692,47
5,0.003444,0.000061,0.002298,0.000013,6,{'kneighborsregressor__n_neighbors': 6},-16.999700,-50.613887,-24.167563,-10.109697,-29.090440,-26.196257,13.799033,45
6,0.004929,0.003024,0.002314,0.000030,7,{'kneighborsregressor__n_neighbors': 7},-15.410640,-50.606670,-24.115023,-10.127901,-29.033526,-25.858752,14.015719,39
7,0.003450,0.000059,0.002294,0.000019,8,{'kneighborsregressor__n_neighbors': 8},-14.163413,-50.621519,-24.086965,-10.124605,-28.888691,-25.577038,14.208884,36
8,0.003445,0.000060,0.002300,0.000016,9,{'kneighborsregressor__n_neighbors': 9},-13.209126,-50.635571,-24.069593,-12.545141,-26.902725,-25.472431,13.818110,33
9,0.003415,0.000046,0.002350,0.000107,10,{'kneighborsregressor__n_neighbors': 10},-12.511094,-50.808438,-24.033708,-11.979642,-26.917104,-25.249997,14.112934,30


In [ ]:
players_min = players_tuned.best_params_
players_min

{'kneighborsregressor__n_neighbors': 40}

In [ ]:
players_best_RMSPE = -players_tuned.best_score_
players_best_RMSPE

np.float64(24.74677992806886)

In [ ]:
knn = KNeighborsRegressor(n_neighbors = 12)
player_pipe = make_pipeline(players_preprocessor, knn)
player_pipe.fit(players[["experience"]], players[["played_hours"]])

print(players_predictions)

players_predictions = players.assign(prediction_all = player_pipe.predict(players[["experience"]]))
players_predictions

     experience  subscribe                                        hashedEmail  \
101           2          1  25879aecc205544bc6505f9faf768356e0a3b712605730...   
51            1          1  b622593d2ef8b337dc554acb307d04a88114f2bf453b18...   
146           4          1  5669c0f4b50dbfe3e2f851e4bb89fa43c55cd1f71ba362...   
153           0          1  0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...   
106           1          0  63139b524e44d156daf265875fb8078bb2b24d6b7520a3...   
59            3          1  ca20f724571080b997e0efa874b9611e9f280c1af5f68f...   
161           3          0  f174555ae3ff613d5d0b98f83f26f689619d92ef272f28...   
167           0          0  42eafe96ed5c1684e3b5cc614d1b01a117173d3ec6898a...   
193           2          0  d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...   
88            0          1  a1094a440899e69e804f888f44d1154ec0b7675d05d977...   
29            3          0  951e54f7376e2b2f0915e9e3646c701af4a2fe839385b1...   
130           2          1  

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName,prediction_all
0,4,1,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN,0.491667
1,3,1,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN,0.625000
2,3,0,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN,0.625000
3,2,1,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN,4.475000
4,1,1,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN,19.208333
...,...,...,...,...,...,...,...,...,...,...
191,2,1,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN,4.475000
192,3,0,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN,0.625000
193,2,0,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN,4.475000
194,2,0,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN,4.475000


In [ ]:

players_plot = (alt.Chart(players_predictions).mark_circle(opacity=0.7).encode(
    x=alt.X("experience").title("Experience").scale(zero=False),
    y=alt.Y("played_hours").title("played_hours").scale(zero=False)
)+
alt.Chart(
    players_predictions,
    title= "K=12"
).mark_line(
    color="black"
).encode(
    x="experience",
    y="prediction_all"
))
players_plot

alt.LayerChart(...)

In [ ]:
# Train only on the training dataset (includes cross validation), predict on the whole dataset